# First time seen remote named pipe
This detection excludes known namped pipes accessible remotely and notify on newly observed ones, may help to detect lateral movement and remote exec using named pipes

## Rule Content
```
- title: First time seen remote named pipe
  id: 52d8b0c6-53d6-439a-9e41-52ad442ad9ad
  description: This detection excludes known namped pipes accessible remotely and
    notify on newly observed ones, may help to detect lateral movement and remote
    exec using named pipes
  author: Samir Bousseaden
  references:
  - https://twitter.com/menasec1/status/1104489274387451904
  tags:
  - attack.lateral_movement
  - attack.t1077
  logsource:
    product: windows
    service: security
    description: The advanced audit policy setting "Object Access > Audit Detailed
      File Share" must be configured for Success/Failure
    category: null
  detection:
    selection1:
      EventID: 5145
      ShareName: \\*\IPC$
    selection2:
      EventID: 5145
      ShareName: \\*\IPC$
      RelativeTargetName:
      - atsvc
      - samr
      - lsarpc
      - winreg
      - netlogon
      - srvsvc
      - protected_storage
      - wkssvc
      - browser
      - netdfs
    condition: selection1 and not selection2
  falsepositives:
  - update the excluded named pipe to filter out any newly observed legit named pipe
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"5145" AND share_name.keyword:\\*\\IPC$) AND (NOT (event_id:"5145" AND share_name.keyword:\\*\\IPC$ AND share_relative_target_name:("atsvc" OR "samr" OR "lsarpc" OR "winreg" OR "netlogon" OR "srvsvc" OR "protected_storage" OR "wkssvc" OR "browser" OR "netdfs"))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()